# Neural PDE Option Greeks — Full Proposal Run

This notebook reproduces the end-to-end experiment described in the project proposal: generate the full synthetic dataset, train the PINN with boundary and PDE losses, and evaluate price/Greek accuracy plus diagnostic surfaces.


## 1. Overview
This research notebook mirrors the experimental protocol described in the accompanying report. Each stage is self-contained so results can be reproduced or stress-tested under alternative settings.

1. **Environment** – establish paths and execution device.
2. **Configuration** – declare dataset and optimisation hyperparameters.
3. **Data Preparation** – synthesise Black–Scholes samples for all splits.
4. **Model Training** – fit the PINN with adaptive sampling and gradient regularisation.
5. **Validation Diagnostics** – benchmark against analytic Greeks in-distribution.
6. **Out-of-Sample Benchmark** – evaluate generalisation vs. baseline estimators.


In [ ]:
# 0) Environment
import os
import sys
from pathlib import Path
import numpy as np
import torch

import plotly.io as pio

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / 'src').exists():
    for parent in PROJECT_ROOT.parents:
        if (parent / 'src').exists():
            PROJECT_ROOT = parent
            break
    else:
        raise RuntimeError("Could not locate project root containing 'src'.")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")
DATA_DIR = PROJECT_ROOT / 'data'
RESULTS_DIR = PROJECT_ROOT / 'results'
FIGURES_DIR = PROJECT_ROOT / 'figures'
RUN_TAG = 'proposal_full_run'
RUN_RESULTS_DIR = RESULTS_DIR / RUN_TAG
FIG_DIR = FIGURES_DIR / RUN_TAG
RUN_RESULTS_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print('Using device:', device)

pio.renderers.default = 'notebook'


## 2. Experimental Configuration
Hyperparameters are grouped into `CONFIG['data']` and `CONFIG['train']` for clarity. The defaults reproduce the settings from our core experiment; edit in-place to explore ablations or stress scenarios.


In [ ]:
# 1) Configuration
# CONFIG = {
#     'run_name': RUN_TAG,
#     'contract': {
#         'K': 100.0,
#         'T': 2.0,
#         'r': 0.05,
#         't_min': 0.01,
#         't_max': 1.999,
#         's_bounds': (20.0, 200.0),
#         'sigma_bounds': (0.05, 0.6),
#     },
#     'data': {
#         'n_train': 1_000_000,
#         'n_val':   100_000,
#         'n_test':  100_000,
#         'seed':    123,
#     },
#     'train': {
#         'epochs': 100,
#         'lr': 1e-3,
#         'batch_size': 2048,
#         'adaptive_sampling': True,
#         'adaptive_every': 50,
#         'adaptive_points': 10_000,
#         'adaptive_radius': 0.1,
#         'adaptive_eval_samples': 50_000,
#         'use_warmup': True,
#         'warmup_steps': 1_000,
#         'grad_clip': 1.0,
#         'lambda_reg': 0.01,
#         'boundary_weight': 1.0,
#         'boundary_warmup': 10,
#     },
#     'evaluation': {
#         'val_subset': 50_000,
#         'sample_size': 50_000,
#         'mc_paths': 100_000,
#         'surface_grid': 60,
#     },
# }

# 1) Configuration - Quick 20 Epoch Run
CONFIG = {
    'run_name': RUN_TAG,
    'contract': {
        'K': 100.0,
        'T': 2.0,
        'r': 0.05,
        't_min': 0.01,
        't_max': 1.999,
        's_bounds': (20.0, 200.0),
        'sigma_bounds': (0.05, 0.6),
    },
    'data': {
        'n_train': 1_000_000,
        'n_val':   100_000,
        'n_test':  100_000,
        'seed':    123,
    },
    'train': {
        'epochs': 20,                    # Short run for validation
        'lr': 1e-3,                      # Standard rate
        'batch_size': 1024,              # Good gradient estimates
        'adaptive_sampling': True,       # Enable
        'adaptive_every': 10,            # Only at epoch 10 and 20
        'adaptive_points': 8_000,        # Slightly fewer points
        'adaptive_radius': 0.08,         # Focused sampling
        'adaptive_eval_samples': 30_000,  # Faster eval
        'use_warmup': True,              # Keep warmup
        'warmup_steps': 300,             # ~0.3 epochs
        'grad_clip': 1.0,                # Standard clipping
        'lambda_reg': 0.005,             # Balanced smoothing
        'boundary_weight': 1.0,          # Full weight
        'boundary_warmup': 5,            # Ramp up faster (over 5 epochs)
    },
    'evaluation': {
        'val_subset': 20_000,            # Faster validation
        'sample_size': 20_000,           # Faster OOS eval
        'mc_paths': 50_000,              # Reduced for speed
        'surface_grid': 80,              # Lower resolution
    },
}

CONFIG

## 3. Data Preparation
We draw $(S,t,\sigma)$ uniformly over the ranges specified in the configuration and label each sample with the Black–Scholes closed-form price. Time is bounded away from maturity to avoid numerical instability. The generated arrays are persisted to `data/` for auditability.


In [ ]:
# 2) Data generation (train/val/test)
from src.data import generate_dataset
contract = CONFIG['contract']
splits = generate_dataset(
    n_train=CONFIG['data']['n_train'],
    n_val=CONFIG['data']['n_val'],
    n_test=CONFIG['data']['n_test'],
    seed=CONFIG['data']['seed'],
    output_dir=DATA_DIR,
    K=contract['K'],
    T=contract['T'],
    r=contract['r'],
    t_min=contract['t_min'],
    t_max=contract['t_max'],
    s_bounds=contract['s_bounds'],
    sigma_bounds=contract['sigma_bounds'],
)
list(splits.keys())

## 4. Model Training
The training routine applies Adam with warmup, gradient clipping, and adaptive sampling concentrated on high-PDE-residual regions. The gradient penalty weight `lambda_reg` moderates surface smoothness, especially for Δ and Γ.


In [ ]:
# 3) Train PINN
from src.train import train
checkpoint_path = RUN_RESULTS_DIR / 'pinn_checkpoint.pt'
plot_path = FIG_DIR / 'loss_curves.html'
log_path = RUN_RESULTS_DIR / 'training_history.json'
model, history = train(
    num_workers=os.cpu_count(),
    pin_memory=(device.type == 'cuda'),
    epochs=CONFIG['train']['epochs'],
    lr=CONFIG['train']['lr'],
    batch_size=CONFIG['train']['batch_size'],
    data_path=DATA_DIR / 'synthetic_train.npy',
    val_path=DATA_DIR / 'synthetic_val.npy',
    checkpoint_path=checkpoint_path,
    device=device,
    adaptive_sampling=CONFIG['train']['adaptive_sampling'],
    adaptive_every=CONFIG['train']['adaptive_every'],
    adaptive_points=CONFIG['train']['adaptive_points'],
    adaptive_radius=CONFIG['train']['adaptive_radius'],
    adaptive_eval_samples=CONFIG['train']['adaptive_eval_samples'],
    use_warmup=CONFIG['train']['use_warmup'],
    warmup_steps=CONFIG['train']['warmup_steps'],
    grad_clip=CONFIG['train']['grad_clip'],
    save_checkpoint=True,
    plot_losses=True,
    plot_path=plot_path,
    log_path=log_path,
    lambda_reg=CONFIG['train']['lambda_reg'],
    boundary_weight=CONFIG['train']['boundary_weight'],
    boundary_warmup=CONFIG['train']['boundary_warmup'],
)
history[-1] if history else 'Loaded existing checkpoint'

### 4.1 Loss Trajectories
The Plotly dashboards below provide both linear-scale and log-scale views of the loss components so we can verify balance between the data-fit term and the physics residual.


In [ ]:
# 4) Review saved loss curves
from IPython.display import display, HTML
loss_html = FIG_DIR / 'loss_curves.html'
loss_log_html = FIG_DIR / 'loss_curves_log.html'
labels = [
    (loss_html, 'Linear scale'),
    (loss_log_html, 'Log scale (L_price, L_PDE, L_reg, L_boundary)'),
]
for html_path, label in labels:
    if html_path.exists():
        display(HTML(f"<h4>{label}</h4>" + html_path.read_text()))
    else:
        print(f"{html_path.name} not found")


## 5. Validation Diagnostics
Using the validation split, we compute pricing RMSE and Greek MAE relative to analytic Black–Scholes references. This quantifies in-distribution fidelity before moving to out-of-sample tests.


In [ ]:
# 5) Validation metrics on synthetic_val
from src.preprocessing import normalize_inputs, load_normalization_config
from src.utils.black_scholes import bs_greeks

val = np.load(DATA_DIR / 'synthetic_val.npy')
subset = val if len(val) <= CONFIG['evaluation']['val_subset'] else val[np.random.choice(
    len(val), CONFIG['evaluation']['val_subset'], replace=False)]
S_np, t_np, sigma_np, price_np = subset.T
config = load_normalization_config(DATA_DIR)

S = torch.tensor(S_np, dtype=torch.float32, device=device)
t = torch.tensor(t_np, dtype=torch.float32, device=device)
sigma = torch.tensor(sigma_np, dtype=torch.float32, device=device)

features = normalize_inputs(S, t, sigma, config=config)
features = features.detach().clone().requires_grad_(True)

model.eval()
pred = model(features).squeeze()

ones = torch.ones_like(pred)
grad_feats = torch.autograd.grad(
    pred, features, grad_outputs=ones, create_graph=True, retain_graph=True
)[0]

grad_x_norm = grad_feats[:, 0]
grad_tau_norm = grad_feats[:, 1]
grad_sigma_norm = grad_feats[:, 2]

# ✅ FIXED: Compute second derivative w.r.t. full features tensor
d2_feats = torch.autograd.grad(
    grad_x_norm,
    features,  # Use features, not features[:, 0]
    grad_outputs=torch.ones_like(grad_x_norm),
    create_graph=True,
    retain_graph=True,
)[0]
d2V_dx_norm2 = d2_feats[:, 0]  # Extract x-x component

# Chain rule scaling factors
x_range = max(config.x_max - config.x_min, 1e-6)
tau_range = max(config.tau_range, 1e-6)
sigma_range = max(config.sigma_range, 1e-6)

dx_norm_dx = 2.0 / x_range
dtau_norm_dtau = 2.0 / tau_range
dsigma_norm_dsigma = 2.0 / sigma_range

# Compute Greeks with chain rule
dV_dx = grad_x_norm * dx_norm_dx
delta = dV_dx / S

d2V_dx2 = d2V_dx_norm2 * (dx_norm_dx**2)
gamma = d2V_dx2 * (1.0 / (S**2)) + dV_dx * (-1.0 / (S**2))

theta = -(grad_tau_norm * dtau_norm_dtau)
vega = grad_sigma_norm * dsigma_norm_dsigma

# Convert to numpy
with torch.no_grad():
    pred_np = pred.detach().cpu().numpy()
    delta_np = delta.detach().cpu().numpy()
    gamma_np = gamma.detach().cpu().numpy()
    theta_np = theta.detach().cpu().numpy()
    vega_np = vega.detach().cpu().numpy()

# Compute analytic Greeks and metrics
analytic = bs_greeks(S_np, config.K, config.T, t_np, sigma_np, config.r)
tau_np = np.clip(config.T - t_np, 1e-6, None)
rho_pred_np = tau_np * (S_np * delta_np - pred_np)

metrics = {
    'price_rmse': float(np.sqrt(np.mean((pred_np - price_np) ** 2))),
    'delta_mae': float(np.mean(np.abs(delta_np - analytic['delta']))),
    'gamma_mae': float(np.mean(np.abs(gamma_np - analytic['gamma']))),
    'theta_mae': float(np.mean(np.abs(theta_np - analytic['theta']))),
    'vega_mae': float(np.mean(np.abs(vega_np - analytic['vega']))),
    'rho_mae': float(np.mean(np.abs(rho_pred_np - analytic['rho']))),
}
metrics

## 6. Out-of-Sample Benchmark
We invoke the evaluation utility to benchmark the PINN against finite-difference and Monte Carlo baselines on held-out data. The function logs metrics and exports interactive plots (2D overlays, error histograms, and 3D surfaces for price, Δ, Γ, Θ, ν, and ρ).

In [ ]:
# 6) Out-of-sample evaluation & visuals
from src.test import evaluate_oos
import json

oos_metrics = evaluate_oos(
    data_path=DATA_DIR / 'synthetic_test.npy',
    model_path=checkpoint_path,
    device=device,
    sample_size=CONFIG['evaluation']['sample_size'],
    mc_paths=CONFIG['evaluation']['mc_paths'],
    seed=CONFIG['data']['seed'],
    fig_dir=FIG_DIR / 'oos',
    surface_grid=CONFIG['evaluation']['surface_grid'],
)
with open(RUN_RESULTS_DIR / 'oos_metrics.json', 'w', encoding='utf-8') as fp:
    json.dump(oos_metrics, fp, indent=2)
oos_metrics

In [ ]:
# 7) Browse generated artifacts
from IPython.display import display, Markdown
import json

display(Markdown('### Training History (JSON excerpt)'))
history_path = RUN_RESULTS_DIR / 'training_history.json'
if history_path.exists():
    with open(history_path) as fp:
        hist = json.load(fp)
    display(hist[-3:])
else:
    print('No training history found.')

display(Markdown('### Saved Loss Curves'))
loss_plot = FIG_DIR / 'loss_curves.html'
loss_log_plot = FIG_DIR / 'loss_curves_log.html'
links = []
labels = [
    (loss_plot, 'Linear scale'),
    (loss_log_plot, 'Log scale (L_price, L_PDE, L_reg, L_boundary)'),
]
for html_path, label in labels:
    if html_path.exists():
        rel_path = html_path.relative_to(PROJECT_ROOT)
        links.append(f"- [{label}]({rel_path.as_posix()})")
    else:
        print(f'{html_path.name} not found.')
if links:
    display(Markdown('\n'.join(links)))


display(Markdown('### Out-of-Sample Figures'))
oos_dir = FIG_DIR / 'oos'
if oos_dir.exists():
    html_links = []
    for html in sorted(oos_dir.glob('*.html')):
        rel_path = html.relative_to(PROJECT_ROOT)
        html_links.append(f"- [{html.name}]({rel_path.as_posix()})")
    if html_links:
        display(Markdown('\n'.join(html_links)))
    else:
        print('No OOS HTML files found.')
else:
    print('No OOS figures found.')

display(Markdown('### Latest OOS Metrics'))
oos_metrics_path = RUN_RESULTS_DIR / 'oos_metrics.json'
if oos_metrics_path.exists():
    with open(oos_metrics_path) as fp:
        display(json.load(fp))
else:
    print('OOS metrics JSON not found.')

## 7. Findings
- Run tag: `proposal_full_run` (proposal-scale dataset: 1M train / 100k val / 100k test).
- Training history stored under `results/proposal_full_run`; loss dashboards live in `figures/proposal_full_run`.
- Validation metrics computed on a 50,000-sample subset of the validation split.
- Out-of-sample diagnostics (metrics JSON and interactive surfaces) are saved in `results/proposal_full_run` and `figures/proposal_full_run/oos`.
